In [ ]:
In this notebook, I use the Yelp api to scrape coffeeshop data from Yelp

In [19]:
import requests
import json
import pandas as pd

In [2]:
api_key = 'sg_yNfXjapD7bhpfSdzBO9hQdlueM-fN-vqhejxBE7c-KMWuFK-5_pTktl02IMtqm-AhImmAGFTRkCefl_xft8953TVwzJCnyVzKGRSmbCLMuNCxUm37sdpH07fYXnYx'
headers = {'Authorization':'Bearer %s' % api_key}

A simple example

In [85]:
url = 'https://api.yelp.com/v3/businesses/search'
params = {'categories':'coffee','latitude':'40.717254','longitude':'-73.995615','limit':'50','offset':'40','radius':'300'}
params2 = {'categories':'coffee','latitude':'40.717254','longitude':'-73.995615','limit':'50','offset':'35','radius':'300'}

In [86]:
req=requests.get(url,params=params,headers=headers)

In [87]:
parsed = json.loads(req.text)
len(parsed['businesses'])

0

In [16]:
req2=requests.get(url,params=params2,headers=headers)
parsed = json.loads(req.text)
len(parsed["businesses"])

39

In [25]:
parsed["businesses"]

[{'id': 'K5OWsf2u4GulpkW2-QLHrg',
  'alias': 'lucky-seven-bakery-new-york',
  'name': 'Lucky Seven bakery',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/sbGvfurzFpPrQw8leWv6iw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/lucky-seven-bakery-new-york?adjust_creative=CGRkejdGiNtvX-lhcOFUZQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=CGRkejdGiNtvX-lhcOFUZQ',
  'review_count': 1,
  'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'},
   {'alias': 'dimsum', 'title': 'Dim Sum'},
   {'alias': 'bakeries', 'title': 'Bakeries'}],
  'rating': 4.0,
  'coordinates': {'latitude': 40.71507, 'longitude': -73.99746},
  'transactions': [],
  'location': {'address1': '47 Bayard St',
   'address2': '',
   'address3': None,
   'city': 'New York',
   'zip_code': '10013',
   'country': 'US',
   'state': 'NY',
   'display_address': ['47 Bayard St', 'New York, NY 10013']},
  'phone': '',
  'display_phone': '',
  'distance': 287.12232459034004},
 {

In [39]:

def keyindictionary(dictionary,key):
    currdict = dictionary
    for i in range(0,len(key)):
        if type(currdict) is not dict:
            return None
        if key[i] in currdict:
            currdict = currdict[key[i]]
        else:
            return None
    return currdict
    
    
def getkeyvaluelist(dictionary,keys):
    result = []
    for key in keys:
        result.append(keyindictionary(dictionary,key))
    return(result)
            

In [67]:
businesslist = []
keystrings = ["id","name","alias","is_closed","categories","review_count","price","rating","transactions"]
keys = [[key] for key in keystrings]
keys.extend([['coordinates','latitude'],['coordinates','longitude']])
test = getkeyvaluelist(parsed["businesses"][0],keys)
for business in parsed["businesses"]:
    businesslist.append(getkeyvaluelist(business,keys))


In [68]:
print(parsed["businesses"][0])
print(test)

{'id': 'K5OWsf2u4GulpkW2-QLHrg', 'alias': 'lucky-seven-bakery-new-york', 'name': 'Lucky Seven bakery', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/sbGvfurzFpPrQw8leWv6iw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/lucky-seven-bakery-new-york?adjust_creative=CGRkejdGiNtvX-lhcOFUZQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=CGRkejdGiNtvX-lhcOFUZQ', 'review_count': 1, 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'}, {'alias': 'dimsum', 'title': 'Dim Sum'}, {'alias': 'bakeries', 'title': 'Bakeries'}], 'rating': 4.0, 'coordinates': {'latitude': 40.71507, 'longitude': -73.99746}, 'transactions': [], 'location': {'address1': '47 Bayard St', 'address2': '', 'address3': None, 'city': 'New York', 'zip_code': '10013', 'country': 'US', 'state': 'NY', 'display_address': ['47 Bayard St', 'New York, NY 10013']}, 'phone': '', 'display_phone': '', 'distance': 287.12232459034004}
['K5OWsf2u4GulpkW2-QLHrg', 'Lucky Seven bakery', 'lucky-sev

In [69]:
businessdf = pd.DataFrame(businesslist)
keystrings.extend(['latitude','longitude'])
businessdf.columns = keystrings

In [100]:
#Now get all this businesses up to the limit of 1000 at a particular location.  
#This function makes search requests until there are no additional businesses found in an area
def searchlocalbustolimit(headers,latstring,longstring,radiusstring,catstring):
    
    businesslist = []
    url = 'https://api.yelp.com/v3/businesses/search'
    offset = 0
    params = {'categories':catstring,'latitude':latstring,'longitude':longstring,'limit':'50','offset':str(offset),'radius':radiusstring}
    req = requests.get(url,params=params,headers=headers)
    parsed = json.loads(req.text)
    for business in parsed["businesses"]:
        businesslist.append(getkeyvaluelist(business,keys))
    numreturned = len(parsed['businesses'])
    print('total is' + str(parsed['total']))
    
    while numreturned >= 50:
        offset = offset + 50
        params['offset'] = str(offset)
        req=requests.get(url,params=params,headers=headers)
        parsed = json.loads(req.text)
        numreturned = len(parsed['businesses'])
        for business in parsed["businesses"]:
            businesslist.append(getkeyvaluelist(business,keys))
    
    return businesslist

        

In [104]:
params = {'categories':'coffee','latitude':'40.717254','longitude':'-73.995615','limit':'50','offset':'40','radius':'300'}

test = searchlocalbustolimit(headers,'40.717254','-73.995615','500','coffee')

total is79


In [105]:
len(test)

79

In [112]:
#Now loop over relevant latitudes and longitudes to get coffeeshops across manhattan
businesslist = []
coords = [[40.717254, -73.995615],
[40.733255, -73.997675],
[40.749058, -73.977247],
[40.766486, -73.994928],
[40.765897, -73.967806],
[40.791899, -73.977591],
[40.778442, -73.953129]
]

for [lat,long] in coords:
    bcurrentloc = searchlocalbustolimit(headers,lat,long,'2000','coffee')
    businesslist.extend(bcurrentloc)

total is609
total is812
total is739
total is504
total is521
total is136
total is232


In [114]:
businessdf = pd.DataFrame(businesslist)
businessdf.columns = keystrings

In [117]:
businessdf.to_csv('mhcoffeeshops_6_4_20.csv',index=False)


In [ ]:
allcoffee = businessdf.drop_duplicates(subset = 'id',keep = 'first')
allcoffee.to_csv('mhcoffeeshops_6_4_20_noduplicates.csv',index=False)